# Project Report I: Data Cleaning

**Report Brief**<br>
In this progress report, I'm hoping to do as much of the data processing as I can, although I don't expect to finish. The breakdown of my project likely means that the heaviest proportion of work will fall on data cleaning/organization, and then EDA, with visualization and analysis being the least intensive section. This is because my project is product-centric--first and foremost, I'm trying to make a *thing* that can classify data.
___
**Goals**

Here's a list of things I'm hoping to accomplish in the first portion of my project:
1. Fix the Markdown formatting for my README and other files
2. Add data sample to GitHub
3. read in data from different folders/sources
4. create a sample DF with a few columns of useful data
    1. text
    2. folder it is in
    3. length of email
    4. to/from
5. visualize some of the data with a few statistical tests or plots

B and D will likely be fairly difficult, and just reading in the data isn't easy, so I'm really aiming to accomplish 1, 2, 3, and 4. Lets see how it goes!
___
**Sharing Data**<br>
In regards to sharing data, there is a bit of a problem in that the source from which I took my data (http://www.cs.cmu.edu/~enron/) doesn't make mention of any type of licensing. I'm going to need to look into this further, but I'm tentatively planning on my data sample being just one (anonymized) person's emails, since this is the most natural data member of the corpus. This will clearly demonstrate the shape and scope of the corpus without taking up too much space or compromising privacy.

In [1]:
import nltk
from nltk.corpus import PlaintextCorpusReader as cr
import pandas as pd
import numpy as np

In [2]:
corpus_root = 'C:/Users/Brett/Desktop/Enron_Emails/maildir/'
all_allen_p = 'allen-p/all_documents'
all_files = cr(corpus_root + all_allen_p, ".*")
print(len(all_files.fileids()))
#needs a way to go one in, regardless of name
#plaintext reader?

628


In [3]:
all_files.words(all_files.fileids()[0])

['Message', '-', 'ID', ':', '<', '29790972', '.', ...]